# Collecting Anomalies

In [ ]:
import feedparser
import pprint
from datetime import datetime, timedelta, date
import pandas as pd
from tqdm import tqdm
import requests
import time
import numpy as np
from market_utils import MarketUtilities

# Display all columns
pd.set_option('display.max_columns', None)

# Display numbers in full (no scientific notation)
pd.set_option('display.float_format', '{:.4f}'.format)

In [ ]:
url = "http://www.nasdaqtrader.com/rss.aspx?feed=tradehalts&haltdate={mmddyyyy}"
def get_halt_data(start_date: datetime.date, end_date: datetime.date, halt_codes: list) -> pd.DataFrame:
    # create empty data structure
    data_lst = []
    
    # Calculate the number of days in the range
    num_days = (end_date - start_date).days + 1
    
    # Loop through the date range with a progress bar
    for i in tqdm(range(num_days), desc="Processing dates"):
        current_date = start_date + timedelta(days=i)

        if i % 100 == 0:
            time.sleep(2)
        
        # parse URL
        str_date = current_date.strftime("%m%d%Y")
        formatted_url = url.format(mmddyyyy=str_date)

        try:
            # Fetch the feed with a timeout
            response = requests.get(formatted_url, timeout=10)
            response.raise_for_status()  # Raise an error for bad status codes
        
            # Parse the feed content
            feed = feedparser.parse(response.content)
        except requests.exceptions.Timeout:
            print('The request timed ou, current_date =', str_date)
            return pd.DataFrame(data_lst)
        except requests.exceptions.RequestException as e:
            print(f'An error occurred: {e}, current_date', str_date)
            return pd.DataFrame(data_lst)

        # check if there are any actual entries
        if len(feed.entries) > 0:
            # iterate through feed entries (articles)
            for entry in feed.entries:
                # only look for specified halt codes
                # continue otherwise
                if halt_codes is not None and entry.ndaq_reasoncode not in halt_codes:
                    continue 
                    
                # empty data entry
                data_entry = {}
                data_entry["halt_date"] = entry.ndaq_haltdate
                data_entry["resumption_date"] = entry.ndaq_resumptiondate
                data_entry["symbol"] = entry.title
                data_entry["issue_name"] = entry.ndaq_issuename
                data_entry["mkt_center"] = entry.ndaq_mkt
                data_entry["halt_code"] = entry.ndaq_reasoncode
                data_entry["halt_time"] = entry.ndaq_halttime
                data_entry["resumption_quote_time"] = entry.ndaq_resumptionquotetime
                data_entry["resumption_trade_time"] = entry.ndaq_resumptiontradetime

                # append to list
                data_lst.append(data_entry)
    
    return pd.DataFrame(data_lst)

In [ ]:
run_collection = False

Dates already covered: ``date(2015, 1, 1)`` to ``date(2025, 1, 15)``

In [ ]:
if run_collection:
    halt_df = get_halt_data(date(2024, 6, 19), date(2025, 1, 15), None)
    halt_df.to_csv("test_data/halt_records.csv", mode='a', index=False, header=False)

In [ ]:
halt_df = pd.read_csv("test_data/halt_records.csv")

In [ ]:
halt_df

In [ ]:
halt_df.halt_code.unique()

In [ ]:
halt_df[halt_df.halt_code == 'H11']

In [ ]:
nsdq_sorted = halt_df[["halt_date", 
         "symbol", 
         "halt_code"]].groupby(["halt_date", 
                                "symbol", 
                                "halt_code"]).value_counts().reset_index().rename(columns={0:"count"}).sort_values("count", ascending=False)

In [ ]:
nsdq_sorted['halt_date'] = pd.to_datetime(nsdq_sorted['halt_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

In [ ]:
nsdq_sorted.sort_values("halt_date")

In [ ]:
nsdq_sorted[nsdq_sorted["count"] >= 5].sort_values("count", ascending=False)[:50]

### NYSE Trade Halt Data

In [ ]:
nyse_halts = pd.read_csv("test_data/nyse_halts.csv").rename(columns={'Halt Date': 'halt_date', 
                                                                                 'Symbol': 'symbol',
                                                                                 'Reason': 'halt_code'})

In [ ]:
nyse_sorted = nyse_halts[["halt_date", 
                          "symbol", 
                          "halt_code"]].groupby(["halt_date", 
                                                 "symbol", 
                                                 "halt_code"]).value_counts().reset_index().rename(columns={0:"count"}).sort_values("count", ascending=False)

In [ ]:
nyse_sorted.halt_code.unique()

In [ ]:
nyse_sorted[nyse_sorted["count"] > 5].sort_values("count", ascending=False)[:50]

### Combining the Two Halt Datasets

In [ ]:
stacked_df = pd.concat([nsdq_sorted, nyse_sorted], ignore_index=True)

In [ ]:
stacked_dedup = stacked_df.drop_duplicates(subset=['halt_date', 'symbol'], keep='first').sort_values('halt_date').reset_index(drop=True)

In [ ]:
# stacked_dedup.to_csv('stacked_dedup_halts.csv')

In [ ]:
stacked_dedup[stacked_dedup.symbol == 'EVLO']

**TODO:** Filter out irrelevant halt codes.

## Query WRDS database

There will be multiple steps to this. We will want 
1. The opening and closing price 15 days before and after the date of the LULD pause
2. The opening and closing price the day of the LULD pause
3. The trade price that triggered the LULD pause (last step)

We can find interday anomalies using 1 and 2. However, for intraday anomalies we will need to determine 3 and possibly some other metric for quantifying average price throughout the trade day.

Since interday anomalies will likely be easier to determine, we will develop the necessary queries for those first.

#### Preliminaries and Test Data

In [ ]:
# imports
import pandas_market_calendars as mcal
import wrds
import yfinance as yf

In [ ]:
mkt_utils = MarketUtilities(wrds_username='audreymcmillion')
db = mkt_utils.wrds_db

In [ ]:
# test mkt_utils function: get_after_date
mkt_utils.get_after_date('2024-07-08', 15)

The below function fetches the industry from the ``yfinance`` API:

In [ ]:
# test mkt_utils function: get_industry_data
mkt_utils.get_industry_data(["ENDP", "AAPL", "TSLA", "MSFT"])

Use the above to get the details.

We will utilize the standard deviation of daily returns to quantify volatility. Additionally, we will add a VIX score to each row when collecting the data.

In [ ]:
# query for before and after average prices
before_after_query = """
    WITH price_details AS (
        SELECT  b.ticker,
                a.permno,
                a.dlycaldt,
                a.dlyopen,
                coalesce(a.dlyclose, a.dlyprc) AS dlyclose,
                a.dlyhigh,
                a.dlylow,
                a.dlynumtrd,
                a.dlyvol,
                (a.dlyopen - a.dlyclose) as daily_return
        FROM crsp.dsf_v2 a
        JOIN crsp.dsenames AS b
        ON a.permno = b.permno
        AND b.ticker IN ({symbol_lst})
        AND a.dlycaldt BETWEEN date('{start_dt}') AND date('{end_dt}')
        AND date('{current_dt}') BETWEEN b.namedt AND b.nameendt
    ),

    days_before AS (
        SELECT ticker,
               MIN(dlycaldt) AS before_date,
               AVG(dlyopen) AS before_avg_open,
               AVG(dlyclose) AS before_avg_close,
               AVG(dlyhigh) AS before_avg_dlyhigh,
               AVG(dlylow) AS before_avg_dlylow,
               AVG(dlynumtrd) AS before_avg_dlynumtrd,
               AVG(dlyvol) AS before_avg_dlyvol
        FROM price_details
        WHERE dlycaldt BETWEEN date('{start_dt}') AND (date('{current_dt}') - INTERVAL '1 day')
        GROUP BY ticker
    ),

    current_days AS (
        SELECT ticker,
               MIN(dlycaldt) AS current_date,
               AVG(dlyopen) AS open_pr,
               AVG(dlyclose) AS close_pr,
               AVG(dlyhigh) AS dlyhigh,
               AVG(dlylow) AS dlylow,
               AVG(dlynumtrd) AS dlynumtrd,
               AVG(dlyvol) AS dlyvol
        FROM price_details
        WHERE dlycaldt = date('{current_dt}')
        GROUP BY ticker
    ),

    days_after AS (
        SELECT ticker,
               MAX(dlycaldt) AS after_date,
               AVG(dlyopen) AS after_avg_open,
               AVG(dlyclose) AS after_avg_close,
               AVG(dlyhigh) AS after_avg_dlyhigh,
               AVG(dlylow) AS after_avg_dlylow,
               AVG(dlynumtrd) AS after_avg_dlynumtrd,
               AVG(dlyvol) AS after_avg_dlyvol
        FROM price_details
        WHERE dlycaldt BETWEEN (date('{current_dt}') + INTERVAL '1 day') AND date('{end_dt}')
        GROUP BY ticker
    )
    
    SELECT coalesce(c.ticker, a.ticker, b.ticker) AS ticker,
           COALESCE(c.current_date, TO_DATE('{current_dt}', 'YYY-MM-DD')) AS current_date,
           COALESCE(b.before_date, TO_DATE('{start_dt}', 'YYY-MM-DD')) AS before_date,
           COALESCE(a.after_date, TO_DATE('{end_dt}', 'YYYY-MM-DD')) AS after_date,
           c.open_pr,
           c.close_pr,
           c.dlyhigh,
           c.dlylow,
           c.dlynumtrd,
           c.dlyvol,
           b.before_avg_open,
           b.before_avg_close,
           b.before_avg_dlyhigh,
           b.before_avg_dlylow,
           b.before_avg_dlynumtrd,
           b.before_avg_dlyvol,
           a.after_avg_open,
           a.after_avg_close,
           a.after_avg_dlyhigh,
           a.after_avg_dlylow,
           a.after_avg_dlynumtrd,
           a.after_avg_dlyvol,
           (SELECT stddev(daily_return) FROM price_details) AS vlty_estimate
    FROM current_days c
    FULL OUTER JOIN days_before b
    ON (c.ticker = b.ticker)
    FULL OUTER JOIN days_after a
    ON (c.ticker = a.ticker)
"""

In [ ]:
import yfinance as yf 

max_date = datetime.strptime('2023-12-29', '%Y-%m-%d')
output_file = "before_after_results.csv"
vix_ticker = yf.Ticker("^VIX") 

i = 0
for current_date in tqdm(stacked_dedup.halt_date.unique()):
    print(current_date)

    # check if valid date
    if datetime.strptime(current_date, '%Y-%m-%d') >= max_date:
        continue
 
    # get subset of LULP 
    subset = stacked_dedup[(stacked_dedup.halt_date == current_date) \
                           & ((stacked_dedup.halt_code == 'LUDP') | (stacked_dedup.halt_code == 'LULD pause')) ]

    # get relevant symbols
    subset_symbols = subset.symbol.to_list()

    if len(subset_symbols) == 0:
        continue

    # get string version of symbols
    stringed_symbols = "', '".join(subset_symbols)
    stringed_symbols = f"'{stringed_symbols}'".replace("''", "'")
    

    # get date strings
    before_date = mkt_utils.get_before_date(current_date, 15)
    after_date = mkt_utils.get_after_date(current_date, 15)

    # check if valid date
    if datetime.strptime(after_date, '%Y-%m-%d') >= max_date:
        continue
    
    # query using extracted values
    before_after_result = db.raw_sql(before_after_query.format(start_dt = before_date,
                                     current_dt = current_date,
                                     end_dt = after_date,
                                     symbol_lst = stringed_symbols))

    # get VIX close
    try:
        before_after_result["vix_close"] = vix_ticker.history(start=current_date, end=get_after_date(current_date, 1))['Close'].iloc[-1]
    except:
        before_after_result["vix_close"] = None

    # get sector data from yfinance
    # industry_data =  get_industry_data(subset_symbols)

    # merge with before_after_result dataframe
    # before_after_result = before_after_result.merge(industry_data, how='left', left_on='ticker', right_on='symbol')

    # write to CSV
    if i == 0:
        # Write the header in the first iteration
        before_after_result.to_csv(output_file, mode='w', index=False)
    else:
        # Append without writing the header in subsequent iterations
        before_after_result.to_csv(output_file, mode='a', index=False, header=False)
        
    i += 1

#### Read in Results

In [ ]:
results = pd.read_csv("test_data/before_after_results.csv")

In [ ]:
results.columns

In [ ]:
results[results.ticker == 'EVLO']

In [ ]:
import yfinance as yf 

# List of stock symbols
symbols = ["AAPL", "GOOGL", "TSLA", "MSFT"]

# Fetch industry for each symbol
industries = [yf.Ticker(symbol).info.get("industryKey") for symbol in symbols]
sectors = [yf.Ticker(symbol).info.get("sectorKey") for symbol in symbols]
industries_dict = {'symbol': symbols, 'industry_key': industries, 'sector_key': sectors}

In [ ]:
industries_dict

In [ ]:
pd.DataFrame.from_dict(industries_dict)

In [ ]:
nona_results = results.dropna(subset=['ticker', 'current_date', 'before_date', 'after_date', 'open_pr',
       'close_pr', 'dlyhigh', 'dlylow', 'dlynumtrd', 'dlyvol',
       'before_avg_open', 'before_avg_close', 'before_avg_dlyhigh',
       'before_avg_dlylow', 'before_avg_dlynumtrd', 'before_avg_dlyvol',
       'after_avg_open', 'after_avg_close', 'after_avg_dlyhigh',
       'after_avg_dlylow', 'after_avg_dlynumtrd', 'after_avg_dlyvol',
       'vlty_estimate']).reset_index(drop=True)

In [ ]:
nona_results["vix_quantile"] = pd.qcut(nona_results.vix_close, 10, labels=False)
nona_results["vlty_quantile"] = pd.qcut(nona_results.vlty_estimate, 10, labels=False)

**Volatility estimates**

In [ ]:
nona_results["vlty_estimate"].describe()

In [ ]:
nona_results["vix_close"].describe()

#### Explore Extreme Values

In [ ]:
# view columns
nona_results.columns

**Get highs and lows**

In [ ]:
daily_highs = nona_results[["ticker", "current_date", "before_date", "after_date", "dlyhigh", "before_avg_dlyhigh",  
                            "after_avg_dlyhigh", "close_pr", "before_avg_close", "after_avg_close", 'dlyvol', 'before_avg_dlyvol', 'after_avg_dlyvol',
                            'vlty_estimate', 'vix_close', 'vix_quantile', 'vlty_quantile']]
daily_lows = nona_results[["ticker", "current_date", "before_date", "after_date", "dlylow", "before_avg_dlylow",  
                           "after_avg_dlylow", "close_pr", "before_avg_close", "after_avg_close", 'dlyvol', 'before_avg_dlyvol', 'after_avg_dlyvol',
                           'vlty_estimate', 'vix_close', 'vix_quantile', 'vlty_quantile']]

**Evaluate daily highs**

In [ ]:
d_highs_fltd = daily_highs[(daily_highs.before_avg_dlyhigh < daily_highs.dlyhigh) \
                           & (daily_highs.after_avg_dlyhigh < daily_highs.dlyhigh)].reset_index(drop=True)

In [ ]:
d_highs_fltd["before_pct_diff"] = ((d_highs_fltd.dlyhigh - d_highs_fltd.before_avg_dlyhigh)/d_highs_fltd.dlyhigh)*100
d_highs_fltd["after_pct_diff"] = ((d_highs_fltd.dlyhigh - d_highs_fltd.after_avg_dlyhigh)/d_highs_fltd.dlyhigh)*100
d_highs_fltd["bef_aft_diff"] = np.abs(((d_highs_fltd.before_avg_dlyhigh - d_highs_fltd.after_avg_dlyhigh)/d_highs_fltd.before_avg_dlyhigh)*100)

In [ ]:
d_highs_fltd[(d_highs_fltd.bef_aft_diff < 5) & (d_highs_fltd.before_pct_diff > 3)].sort_values(["vlty_quantile", "bef_aft_diff"], ascending=True)

**View how we might determine cutoffs based on the before/after percent differences.**

In [ ]:
d_highs_fltd[(d_highs_fltd.bef_aft_diff < 10)][["vlty_quantile", 
                                               "before_pct_diff", 
                                               "after_pct_diff"]].groupby("vlty_quantile").describe()

**Visualize individual results using the below function:**

In [ ]:
mkt_utils.multiday_chart('GDEF', '2014-09-25')

In [ ]:
mkt_utils.multiday_chart('VVPR', '2020-10-09')

**Evaluate daily lows**

In [ ]:
d_lows_fltd = daily_lows[(daily_lows.before_avg_dlylow > daily_lows.dlylow) \
                         & (daily_lows.after_avg_dlylow > daily_lows.dlylow)].reset_index(drop=True)

In [ ]:
d_lows_fltd

In [ ]:
d_lows_fltd["before_pct_diff"] = ((d_lows_fltd.dlylow - d_lows_fltd.before_avg_dlylow)/d_lows_fltd.dlylow)*100
d_lows_fltd["after_pct_diff"] = ((d_lows_fltd.dlylow - d_lows_fltd.after_avg_dlylow)/d_lows_fltd.dlylow)*100
d_lows_fltd["bef_aft_diff"] = np.abs(((d_lows_fltd.before_avg_dlylow - d_lows_fltd.after_avg_dlylow)/d_lows_fltd.before_avg_dlylow)*100)

In [ ]:
d_lows_fltd

In [ ]:
d_lows_fltd[(d_lows_fltd.before_pct_diff < -40) \
             & (d_lows_fltd.after_pct_diff < -40) \
             & (d_lows_fltd.bef_aft_diff < 10)].sort_values(["bef_aft_diff"], ascending=True)

In [ ]:
mkt_utils.multiday_chart('BRAG', '2022-06-08', high=False)

#### Determine Extreme Values

First, generate cutoff/threshold dictionaries for the high and low dataframes.

In [ ]:
# function to get cutoff dictionaries based on volatility quantiles
def cutoffs_dict(hl_df: pd.DataFrame, pct_diff_quantile: float, bef_aft_quantile: float, max_bef_aft_diff = 10):
    cutoff_dict = {}
    
    for val in sorted(hl_df.vlty_quantile.unique()):
        # get before and after percentages
        bef_percent = np.abs(hl_df[(hl_df.bef_aft_diff < max_bef_aft_diff) 
                             & (hl_df.vlty_quantile == val)]["before_pct_diff"]).quantile(pct_diff_quantile)
        aft_percent = np.abs(hl_df[(hl_df.bef_aft_diff < max_bef_aft_diff) 
                             & (hl_df.vlty_quantile == val)]["after_pct_diff"]).quantile(pct_diff_quantile)

        # get cutoff for bef_aft_diff
        bef_aft_cutoff = hl_df[(hl_df.bef_aft_diff < max_bef_aft_diff) & (hl_df.vlty_quantile == val)]["bef_aft_diff"].quantile(bef_aft_quantile)

        # add to dictionary
        cutoff_dict[val] = {"pct_diff_cutoff": np.mean([bef_percent, aft_percent]), "bef_aft_cutoff": bef_aft_cutoff}

    return cutoff_dict

In [ ]:
high_cutoff_dict = cutoffs_dict(d_highs_fltd, pct_diff_quantile = 0.75, bef_aft_quantile = 0.3)
low_cutoff_dict = cutoffs_dict(d_lows_fltd, pct_diff_quantile = 0.75, bef_aft_quantile = 0.3)

In [ ]:
high_cutoff_dict

Now, use the below function to flag suspected anomalies.

In [ ]:
def flag_anomaly(row, high: bool) -> bool:
    # get relevant values
    row_before = np.abs(row.before_pct_diff)
    row_after = np.abs(row.after_pct_diff)
    row_bef_aft = row.bef_aft_diff
    row_vol = row.vlty_quantile

    # get dictionary values
    if high:
        pct_diff_cutoff = high_cutoff_dict[row_vol]["pct_diff_cutoff"]
        bef_aft_cutoff = high_cutoff_dict[row_vol]["bef_aft_cutoff"]
    else:
        pct_diff_cutoff = low_cutoff_dict[row_vol]["pct_diff_cutoff"]
        bef_aft_cutoff = low_cutoff_dict[row_vol]["bef_aft_cutoff"]

    if (row_before >= pct_diff_cutoff) & (row_after >= pct_diff_cutoff) & (row_bef_aft <= bef_aft_cutoff):
        return True
    else:
        return False

In [ ]:
d_highs_fltd['anomaly_fl'] = d_highs_fltd.apply(lambda row: flag_anomaly(row, high=True), axis=1)
d_lows_fltd['anomaly_fl'] = d_highs_fltd.apply(lambda row: flag_anomaly(row, high=False), axis=1)

In [ ]:
d_highs_fltd.to_csv("high_anomalies.csv", index=False)

In [ ]:
mkt_utils.multiday_chart('PALI', '2023-09-06', high = True, diff_num = 15)

In [ ]:
d_lows_fltd.to_csv("low_anomalies.csv", index=False)

## Testing Open Source Market Data

**Finance database**: https://github.com/JerBouma/FinanceDatabase
- "The FinanceDatabase serves the role of providing anyone with any type of financial product categorization entirely for free. To be able to achieve this, the FinanceDatabase relies on involvement from the community to add, edit and remove tickers over time. This is made easy enough that anyone, even with a lack of coding experience can contribute because of the usage of CSV files that can be manually edited with ease."

In [ ]:
import financedatabase as fd

# Initialize the Equities database
equities = fd.Equities()

In [ ]:
# Use the tickers to obtain data via the Finance Toolkit
telecomunication_services = equities.search(
    industry="Diversified Telecommunication Services",
    country="United States",
    market_cap="Mega Cap",
    exclude_exchanges=True)

In [ ]:
all_symbols = equities.select(exclude_exchanges=True).reset_index()
all_symbols['symbol'] = all_symbols['symbol'].fillna('NA')  # Fill NaNs in 'price' column with 0

In [ ]:
all_symbols[all_symbols.symbol == 'AAPL']

In [ ]:
all_symbols.industry.unique().tolist()

In [ ]:
# all_symbols.drop_duplicates().reset_index(drop=True).to_csv("symbol_details.csv", index=False)

Can we fill in the blank for some of these using Yahoo's API?

In [ ]:
# mkt_utils = MarketUtilities(wrds_username='audreymcmillion')
mkt_utils.get_industry_data(["FTEL", 'IXHL', 'AIRE'])

Get tickers with null sectors from database.

In [ ]:
null_sector_tickers = pd.read_sql("""
with sector_luld as (
	select bar.*, sd.sector, sd.industry_group, sd.industry, sd.market_cap
	from before_after_results bar
	left join symbol_details sd 
	on (bar.ticker) = (sd.symbol)
	order by bar.current_date desc
)

select distinct s.ticker
from sector_luld s
where s.sector is null
""", mkt_utils.sqlite_conn)             

In [ ]:
null_sector_tickers = null_sector_tickers.ticker.to_list()

In [ ]:
len(null_sector_tickers)

In [ ]:
luldnull_sectors = mkt_utils.get_industry_data(null_sector_tickers)

In [ ]:
symbol_nonnull = luldnull_sectors[~luldnull_sectors.industry.isna()].reset_index(drop=True)

In [ ]:
sorted(symbol_nonnull.industry.unique().tolist())

In [ ]:
sorted(symbol_nonnull.sector.unique().tolist())

In [ ]:
all_sector = pd.read_sql("""
	select distinct sd.sector, sd.industry_group, sd.industry, sd.market_cap
	from symbol_details sd  
    where sd.sector is not null
    and sd.industry_group is not null
    and sd.industry is not null
    order by sd.sector
""", mkt_utils.sqlite_conn)            

In [ ]:
sorted(all_sector.industry.unique().tolist())

In [ ]:
sorted(all_sector.sector.unique().tolist())

In [ ]:
industry_mapping = {
    'Capital Markets': 'Capital Markets',
    'Consumer Electronics': 'Household Durables',
    'Software - Infrastructure': 'Software',
    'Building Materials': 'Building Products',
    'Beverages - Non-Alcoholic': 'Beverages',
    'Biotechnology': 'Biotechnology',
    'Specialty Industrial Machinery': 'Machinery',
    'Computer Hardware': 'Technology Hardware, Storage & Peripherals',
    'Real Estate - Development': 'Real Estate Management & Development',
    'Specialty Retail': 'Specialty Retail',
    'Consulting Services': 'Professional Services',
    'Health Information Services': 'Health Care Technology',
    'Integrated Freight & Logistics': 'Air Freight & Logistics',
    'Asset Management': 'Diversified Financial Services',
    'Auto Parts': 'Auto Components',
    'Real Estate Services': 'Real Estate Management & Development',
    'Software - Application': 'Software',
    'Internet Content & Information': 'Interactive Media & Services',
    'Information Technology Services': 'IT Services',
    'Specialty Business Services': 'Professional Services',
    'Conglomerates': 'Industrial Conglomerates',
    'Grocery Stores': 'Food & Staples Retailing',
    'Aerospace & Defense': 'Aerospace & Defense',
    'Advertising Agencies': 'Media',
    'Solar': 'Independent Power and Renewable Electricity Producers',
    'Medical Care Facilities': 'Health Care Providers & Services',
    'Farm Products': 'Food Products',
    'Drug Manufacturers - General': 'Pharmaceuticals',
    'Lumber & Wood Production': 'Paper & Forest Products',
    'Leisure': 'Hotels, Restaurants & Leisure',
    'Auto Manufacturers': 'Automobiles',
    'Waste Management': 'Commercial Services & Supplies',
    'Communication Equipment': 'Communications Equipment',
    'Auto & Truck Dealerships': 'Distributors',
    'Packaged Foods': 'Food Products',
    'Shell Companies': None,
    'Personal Services': 'Diversified Consumer Services',
    'Other Industrial Metals & Mining': 'Metals & Mining',
    'Marine Shipping': 'Marine',
    'Utilities - Renewable': 'Independent Power and Renewable Electricity Producers',
    'Security & Protection Services': 'Commercial Services & Supplies',
    'Electrical Equipment & Parts': 'Electrical Equipment',
    'Electronic Gaming & Multimedia': 'Entertainment',
    'Railroads': 'Road & Rail',
    'Banks - Regional': 'Banks',
    'Gambling': 'Hotels, Restaurants & Leisure',
    'Scientific & Technical Instruments': 'Electronic Equipment, Instruments & Components',
    'Diagnostics & Research': 'Health Care Equipment & Supplies',
    'Drug Manufacturers - Specialty & Generic': 'Pharmaceuticals',
    'Lodging': 'Hotels, Restaurants & Leisure',
    'Electronic Components': 'Electronic Equipment, Instruments & Components',
    'Entertainment': 'Entertainment',
    'Oil & Gas Integrated': 'Oil, Gas & Consumable Fuels',
    'Travel Services': 'Hotels, Restaurants & Leisure',
    'Medical Devices': 'Health Care Equipment & Supplies',
    'Credit Services': 'Consumer Finance',
    'Internet Retail': 'Internet & Direct Marketing Retail',
    'Utilities - Regulated Water': 'Water Utilities'
}

sector_mapping = {
    'Basic Materials': 'Materials',
    'Communication Services': 'Communication Services',
    'Consumer Cyclical': 'Consumer Discretionary',
    'Consumer Defensive': 'Consumer Staples',
    'Energy': 'Energy',
    'Financial Services': 'Financials',
    'Healthcare': 'Health Care',
    'Industrials': 'Industrials',
    'Real Estate': 'Real Estate',
    'Technology': 'Information Technology',
    'Utilities': 'Utilities'
}

In [ ]:
symbol_nonnull['industry'] = symbol_nonnull['industry'].map(industry_mapping)
symbol_nonnull['sector'] = symbol_nonnull['sector'].map(sector_mapping)

In [ ]:
# symbol_nonnull.to_csv("yfinance_sectors.csv", index=False)

Examine the "still nulls"...

In [ ]:
still_nulls = luldnull_sectors[luldnull_sectors.industry.isna()].symbol.unique()

In [ ]:
still_nulls

Another option, **TODO** when I get home tonight.

In [ ]:
from sec_api import MappingApi
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Initialize the MappingApi with your API key
mapping_api = MappingApi(api_key=os.getenv("SEC_API_KEY"))

# List of tickers
tickers = ['BRSH', 'HRYU', 'PXDT', 'BSGA','REVE']

# Dictionary to store the results
ticker_info = {'symbol':[], 'sector':[], 'industry':[]}

for ticker in tickers:   
    ticker_info['symbol'].append(ticker) 
    
    # Resolve ticker to get company details
    company_details = mapping_api.resolve('ticker', ticker)
    if company_details:
        # Extract the first matching result
        details = company_details[0]
        ticker_info['sector'].append(details.get('sector'))
        ticker_info['industry'].append(details.get('industry'))
    else:
        ticker_info['sector'].append(None)
        ticker_info['industry'].append(None)

In [ ]:
pd.DataFrame(ticker_info)

**NOTES**:  
WBUY	Internet & Direct Marketing Retail	Consumer Discretionary  
NTBL	Biotechnology	Health Care  
DMK	Pharmaceuticals	Health Care  